## 38.9 PROJECT. Исследование поведения пользователей

**Постановка задачи**
Провести анализ поведения пользователей мобильной игры Quiz Freeze.
Необходимо проверить:
- есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;
- различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.
Выполнять задание необходимо на основе данных пользователей, которые зарегистрировались в 2018 году (с 1 января по 31 декабря 2018 года включительно).

**Исходные данные**
Два файла - таблицы в формате csv

**Обнаруженные несоотетствия**
В одной из таблиц были события с некоррекнтой датой. Такие события были удалены в ходе предобработки данных.

#### Импортируем PANDAS и исходные данные.

In [2]:
import pandas as pd
events_initial = pd.read_csv('C:/New_life/DataAnalytics/Python/38_event_purchase/7_4_Events.csv', sep=',')
events = events_initial.copy()
purchase_initial = pd.read_csv('C:/New_life/DataAnalytics/Python/38_event_purchase/purchase.csv', sep=',')
purchase = purchase_initial.copy()

#### ПРЕДОБРАБОТКА ДАННЫХ
1) Убрираем в таблицах events строки с некорректными датой/временем. Переводим дату/время в формат datetime.
    При попытке перевести данные колонки start_time в таблице events в формат datetime были обнаружены несколько некорректных значений даты-времени. С учетом того, что таких значений крайне мало, а также что они явно не относятся к 2018 году, строки, содержащие такие значения были удалены.
2) Оставляем в таблицах только события пользоваетелй с регистрацией в 2018 году

Убрираем в таблице events строки с некорректными датами.

In [3]:
print("Посмотрим на исходную таблицу events:\n", events.head())    # Формат колонки с временем событий - 2016-05-11T23:40:55
def remove_t(start_time):   # функция удаления буквы "Т", разделяющей дату и время в исходных данных
    good_time=start_time.replace('T',' ')
    return good_time
events['start_time']=events['start_time'].apply(remove_t)

# При попытке перевести колонку в формат datetime выдавались ошибки, связанные с некорректным значением даты/времени
def find_wrong(start_time):     # функция определения неверных даты-времени (недостаток цифр или дата - 29 февраля 
                                # невискостного года)
    if len(start_time) != 19:
        return 'wrong date time' 
    elif start_time[5:7] == '02' and int(start_time[8:10]) > 28:
        if int(start_time[:4])%4 != 0:
            return 'wrong date time'
        elif int(start_time[:3])%100 == 0 and int(start_time[:3])%400 != 0:
            return 'wrong date time'

events['wrong_start_time']=events['start_time'].apply(find_wrong)   # добавляем колонку-флаг некорректности даты-времени
print("\nCобытия с некорректной датой/временем:\n", events[events['wrong_start_time'] == 'wrong date time'].head())
print("\nCколько событий с некорректной датой/временем? -", events[events['wrong_start_time'] == 'wrong date time'].shape[0])
print("Cколько событий всего? -", events.shape[0])
# Можно удалить событися без нарушения релевантности данных.
events=events[events['wrong_start_time'] != 'wrong date time'] 
events.drop(labels='wrong_start_time',axis=1, inplace=True)

Посмотрим на исходную таблицу events:
       id       event_type selected_level           start_time  tutorial_id  \
0  28903     registration            NaN  2016-05-11T23:40:55          NaN   
1  28904     registration            NaN  2016-05-11T23:49:58          NaN   
2  28905     registration            NaN  2016-05-12T00:53:07          NaN   
3  28906   tutorial_start            NaN  2016-05-12T01:32:20      17562.0   
4  28907  tutorial_finish            NaN  2016-05-12T01:34:53      17562.0   

   user_id  
0    12583  
1    12584  
2    12585  
3    12585  
4    12585  

Cобытия с некорректной датой/временем:
           id      event_type selected_level           start_time  tutorial_id  \
10981  39884    registration            NaN       20162015-09-18          NaN   
19423  48326  tutorial_start            NaN  2017-02-29 00:49:14      22840.0   
19424  48327    level_choice         medium  2017-02-29 02:23:01          NaN   
19425  48328     pack_choice            NaN  2017

Переводим колонку start_time в events в datetime формат 

In [4]:
events['start_time']=pd.to_datetime(events['start_time'])
print("Обработанная таблица  events:\n", events.head())

Обработанная таблица  events:
       id       event_type selected_level          start_time  tutorial_id  \
0  28903     registration            NaN 2016-05-11 23:40:55          NaN   
1  28904     registration            NaN 2016-05-11 23:49:58          NaN   
2  28905     registration            NaN 2016-05-12 00:53:07          NaN   
3  28906   tutorial_start            NaN 2016-05-12 01:32:20      17562.0   
4  28907  tutorial_finish            NaN 2016-05-12 01:34:53      17562.0   

   user_id  
0    12583  
1    12584  
2    12585  
3    12585  
4    12585  


Оставляем в таблице events только события пользоваетелй с регистрацией в 2018 году -> новая таблица events_users_reg2018

In [5]:
users_reg2018=events[(events['start_time'].dt.year == 2018) & (events['event_type'] == 
    'registration')]['user_id'].drop_duplicates()   # перечень пользователей с регстрацией в 2018 году
users_reg2018=pd.DataFrame(data=users_reg2018)   
events_user_reg2018=users_reg2018.merge(events,how='left',on='user_id')
print(events_user_reg2018.head())

   user_id     id       event_type selected_level          start_time  \
0    27832  80308     registration            NaN 2018-01-01 03:48:40   
1    27833  80309     registration            NaN 2018-01-01 04:07:25   
2    27833  80320   tutorial_start            NaN 2018-01-01 17:47:40   
3    27833  80321  tutorial_finish            NaN 2018-01-01 17:50:08   
4    27834  80310     registration            NaN 2018-01-01 08:35:10   

   tutorial_id  
0          NaN  
1          NaN  
2      31508.0  
3      31508.0  
4          NaN  


Переводим колонку event_datetime в purchase в datetime формат 

In [6]:
print("Посмотрим на исходную таблицу purchase:\n", purchase.head())   
purchase['event_datetime']=purchase['event_datetime'].apply(remove_t)
purchase['event_datetime']=pd.to_datetime(purchase['event_datetime'])
print("Обработанная таблица  purchase:\n", purchase.head())

Посмотрим на исходную таблицу purchase:
       id  user_id       event_datetime  amount
0  15674    12584  2016-05-12T10:34:16     100
1  15675    12985  2016-05-13T08:25:56      50
2  15676    12828  2016-05-13T16:33:46      50
3  15677    12598  2016-05-14T01:09:37     150
4  15678    13037  2016-05-14T01:24:46     100
Обработанная таблица  purchase:
       id  user_id      event_datetime  amount
0  15674    12584 2016-05-12 10:34:16     100
1  15675    12985 2016-05-13 08:25:56      50
2  15676    12828 2016-05-13 16:33:46      50
3  15677    12598 2016-05-14 01:09:37     150
4  15678    13037 2016-05-14 01:24:46     100


Оставляем в таблице purchase только события пользоваетелй с регистрацией в 2018 году -> новая таблица purchase_user_reg2018

In [7]:
purchase_user_reg2018=users_reg2018.merge(purchase,how='inner',on='user_id')
print(purchase_user_reg2018.head())

   user_id     id      event_datetime  amount
0    27845  16845 2018-01-03 18:53:43     100
1    27865  16846 2018-01-04 14:46:10     250
2    27884  16854 2018-01-08 19:37:34     150
3    27910  16849 2018-01-07 12:11:34     100
4    27911  16848 2018-01-07 08:19:12      50


#### ПРЕДВАРИТЕЛЬНЫЙ АНАЛИЗ ДАННЫХ

In [8]:
print("Варианты событий:", events['event_type'].unique())

Варианты событий: ['registration' 'tutorial_start' 'tutorial_finish' 'level_choice'
 'pack_choice']


Выделим из таблицы events события типа level_choice

In [9]:
events_selected_level=events_user_reg2018[events_user_reg2018['event_type'] == 'level_choice']   
    #events с событием level_choice
# убедимся что все пользователи выполняли действие level_choice не более одного раза
print("events с событием level_choice:\n", events_selected_level.head())
print("\nВсе пользователи выполняли действие level_choice не более одного раза?   ", 
    events_selected_level.shape[0] == events_selected_level['user_id'].nunique())


events с событием level_choice:
     user_id     id    event_type selected_level          start_time  \
10    27835  80327  level_choice         medium 2018-01-01 20:37:22   
21    27839  80331  level_choice           hard 2018-01-01 22:37:50   
25    27840  80344  level_choice         medium 2018-01-02 05:18:42   
32    27842  80353  level_choice           easy 2018-01-02 08:46:03   
39    27843  80365  level_choice         medium 2018-01-02 14:09:58   

    tutorial_id  
10          NaN  
21          NaN  
25          NaN  
32          NaN  
39          NaN  

Все пользователи выполняли действие level_choice не более одного раза?    True


Убедимся что все пользователи совершали оплату не более одного раза

In [10]:
print("Все пользователи совершали оплату не более одного раза:   ", 
    purchase_user_reg2018.shape[0] == purchase_user_reg2018['user_id'].nunique())

Все пользователи совершали оплату не более одного раза:    True


Посмотрим сколько пользователей совершили level_choice, сколько пользователей сделали оплаты и убедимся что все пользователи, сделавшие оплату, совершили level_choice 

In [11]:
print("Сколько пользователей совершили level_choice? -", events_selected_level.shape[0])
print("Сколько пользователей сделали оплату? -", purchase_user_reg2018.shape[0])
print("Все пользователи, сделавшие оплату, совершили level_choice:   ", 
    purchase_user_reg2018[purchase_user_reg2018['user_id'].isin(events_selected_level['user_id'])]['user_id'].shape[0]
    ==
    purchase_user_reg2018.shape[0])

Сколько пользователей совершили level_choice? - 8342
Сколько пользователей сделали оплату? - 1600
Все пользователи, сделавшие оплату, совершили level_choice:    True


Объеденим таблицы с событиями и с оплатой в единую таблицу для пользователей, совершивших level_choice, и уберем лишние колонки

In [12]:
events_selected_level_new=events_selected_level.rename(columns={'start_time':'level_choice_time'}).drop(
    labels=['id','tutorial_id'],axis=1)
purchase_user_reg2018_new=purchase_user_reg2018.rename(columns={'event_datetime':'purchase_time'}).drop(
    labels=['id','amount'],axis=1)
events_selected_level_purchase=events_selected_level_new.merge(purchase_user_reg2018_new, how='left', on='user_id')
print("Объединенная таблица:\n", events_selected_level_purchase.head())

Объединенная таблица:
    user_id    event_type selected_level   level_choice_time purchase_time
0    27835  level_choice         medium 2018-01-01 20:37:22           NaT
1    27839  level_choice           hard 2018-01-01 22:37:50           NaT
2    27840  level_choice         medium 2018-01-02 05:18:42           NaT
3    27842  level_choice           easy 2018-01-02 08:46:03           NaT
4    27843  level_choice         medium 2018-01-02 14:09:58           NaT


#### ОПРЕДЕЛИМ, ЕСТЬ ЛИ ЗАВИСИМОСТЬ МЕЖДУ ВЫБРАННЫМ УРОВНЕМ СЛОЖНОСТИ И ВЕРОЯТНОСТЬЮ ОПЛАТЫ

Разобъем объединенную таблицу на три таблицы по selected_level

In [13]:
print("Посмотрим, какие имеются selected_level и все ли варианты статистически значимы:")
print(events_selected_level_purchase['selected_level'].value_counts())
easy_level=events_selected_level_purchase[events_selected_level_purchase['selected_level'] == 'easy']
medium_level=events_selected_level_purchase[events_selected_level_purchase['selected_level'] == 'medium']
hard_level=events_selected_level_purchase[events_selected_level_purchase['selected_level'] == 'hard']
print("\nТаблица для уровня easy:\n", easy_level.head())
print("\nТаблица для уровня medium:\n", medium_level.head())
print("\nТаблица для уровня hard:\n", hard_level.head())

Посмотрим, какие имеются selected_level и все ли варианты статистически значимы:
medium    4645
easy      2448
hard      1249
Name: selected_level, dtype: int64

Таблица для уровня easy:
     user_id    event_type selected_level   level_choice_time purchase_time
3     27842  level_choice           easy 2018-01-02 08:46:03           NaT
8     27849  level_choice           easy 2018-01-02 11:53:11           NaT
10    27853  level_choice           easy 2018-01-03 00:39:54           NaT
14    27859  level_choice           easy 2018-01-03 01:11:45           NaT
17    27863  level_choice           easy 2018-01-03 09:31:00           NaT

Таблица для уровня medium:
    user_id    event_type selected_level   level_choice_time purchase_time
0    27835  level_choice         medium 2018-01-01 20:37:22           NaT
2    27840  level_choice         medium 2018-01-02 05:18:42           NaT
4    27843  level_choice         medium 2018-01-02 14:09:58           NaT
6    27846  level_choice         medi

Посчитаем долю оплат в группах с разным selected_level

In [14]:
percent_purchase_easy=easy_level['purchase_time'].count() / easy_level['level_choice_time'].count() 
print("Доля оплативших среди пользователей, выбравших уровень easy:     ", round(percent_purchase_easy, 2))
percent_purchase_medium=medium_level['purchase_time'].count() / medium_level['level_choice_time'].count() 
print("Доля оплативших среди пользователей, выбравших уровень medium:   ", round(percent_purchase_medium, 2))
percent_purchase_hard=hard_level['purchase_time'].count() / hard_level['level_choice_time'].count() 
print("Доля оплативших среди пользователей, выбравших уровень hard:     ", round(percent_purchase_hard, 2))

Доля оплативших среди пользователей, выбравших уровень easy:      0.08
Доля оплативших среди пользователей, выбравших уровень medium:    0.21
Доля оплативших среди пользователей, выбравших уровень hard:      0.35


**ВЫВОД 1:**
Чем сложнее выбранный уровень (selected_level), тем более вероятен факт оплаты.

#### ОПРЕДЕЛИМ, РАЗЛИЧАЕТСЯ ЛИ ВРЕМЕННОЙ ПРОМЕЖУТОК МЕЖДУ РЕГИСТРАЦИЕЙ И ОПЛАТОЙ<br> У ГРУПП ПОЛЬЗОВАТЕЛЕЙ С РАЗНЫМ УРОВНЕМ СЛОЖНОСТИ

Оставим в таблице events только строки с событием registration

In [15]:
events_registration=events_user_reg2018[events_user_reg2018['event_type'] == 'registration']   
    #events с событием registration
print(events_registration.head())

    user_id     id    event_type selected_level          start_time  \
0     27832  80308  registration            NaN 2018-01-01 03:48:40   
1     27833  80309  registration            NaN 2018-01-01 04:07:25   
4     27834  80310  registration            NaN 2018-01-01 08:35:10   
7     27835  80311  registration            NaN 2018-01-01 11:54:47   
12    27836  80312  registration            NaN 2018-01-01 13:28:07   

    tutorial_id  
0           NaN  
1           NaN  
4           NaN  
7           NaN  
12          NaN  


Объеденим таблицы с registration и с оплатой в единую таблицу, и уберем лишние колонки

In [16]:
events_registration_new=events_registration.rename(columns={'start_time':'registration_time'}).drop(axis=1,
    labels=['selected_level', 'id', 'tutorial_id'])
events_registration_purchase=events_registration_new.merge(purchase_user_reg2018_new, how='right', on='user_id')
print(events_registration_purchase.head())

   user_id    event_type   registration_time       purchase_time
0    27845  registration 2018-01-02 01:35:56 2018-01-03 18:53:43
1    27865  registration 2018-01-03 11:14:57 2018-01-04 14:46:10
2    27884  registration 2018-01-04 11:50:43 2018-01-08 19:37:34
3    27910  registration 2018-01-05 10:45:33 2018-01-07 12:11:34
4    27911  registration 2018-01-05 10:48:24 2018-01-07 08:19:12


Посчитаем время, прошедшее с регистрации до оплаты

In [17]:
events_registration_purchase['time_dif'] = events_registration_purchase['purchase_time']- events_registration_purchase[
    'registration_time']
print(events_registration_purchase.head())

   user_id    event_type   registration_time       purchase_time  \
0    27845  registration 2018-01-02 01:35:56 2018-01-03 18:53:43   
1    27865  registration 2018-01-03 11:14:57 2018-01-04 14:46:10   
2    27884  registration 2018-01-04 11:50:43 2018-01-08 19:37:34   
3    27910  registration 2018-01-05 10:45:33 2018-01-07 12:11:34   
4    27911  registration 2018-01-05 10:48:24 2018-01-07 08:19:12   

         time_dif  
0 1 days 17:17:47  
1 1 days 03:31:13  
2 4 days 07:46:51  
3 2 days 01:26:01  
4 1 days 21:30:48  


Разобьем полученную таблицу на три таблицы по selected_level и оценим статистическую значимость

In [18]:
events_registration_purchase_easy=events_registration_purchase[events_registration_purchase['user_id'].isin(
    easy_level['user_id'])]
print("\nТаблица для уровня easy:", "Число строк -   ", events_registration_purchase_easy.shape[0], "\n", 
      events_registration_purchase_easy.head())
events_registration_purchase_medium=events_registration_purchase[events_registration_purchase['user_id'].isin(
    medium_level['user_id'])]
print("\nТаблица для уровня medium:", "Число строк -   ", events_registration_purchase_medium.shape[0], "\n", 
      events_registration_purchase_medium.head())
events_registration_purchase_hard=events_registration_purchase[events_registration_purchase['user_id'].isin(
    hard_level['user_id'])]
print("\nТаблица для уровня hard:", "Число строк -   ", events_registration_purchase_hard.shape[0], "\n", 
      events_registration_purchase_hard.head())


Таблица для уровня easy: Число строк -    189 
     user_id    event_type   registration_time       purchase_time  \
2     27884  registration 2018-01-04 11:50:43 2018-01-08 19:37:34   
20    28090  registration 2018-01-09 19:31:24 2018-01-15 23:42:55   
25    28182  registration 2018-01-11 10:12:20 2018-01-12 02:46:01   
27    28207  registration 2018-01-11 16:27:37 2018-01-12 21:00:24   
31    28247  registration 2018-01-12 10:01:12 2018-01-18 18:32:05   

          time_dif  
2  4 days 07:46:51  
20 6 days 04:11:31  
25 0 days 16:33:41  
27 1 days 04:32:47  
31 6 days 08:30:53  

Таблица для уровня medium: Число строк -    969 
     user_id    event_type   registration_time       purchase_time  \
8     27973  registration 2018-01-06 22:01:44 2018-01-13 21:50:00   
10    27981  registration 2018-01-07 08:09:09 2018-01-07 23:20:25   
13    28010  registration 2018-01-07 22:19:23 2018-01-10 05:32:47   
15    28020  registration 2018-01-08 10:30:32 2018-01-11 21:43:03   
16    28026  r

Посчитаем среднее и медианное время между регистрацией и оплатой для пользователей с разным selected_level

In [19]:
time_diff_easy_mean=events_registration_purchase_easy['time_dif'].mean() 
time_diff_easy_median=events_registration_purchase_easy['time_dif'].median() 
#print(events_registration_purchase_easy['time_dif'].describe())
time_diff_medium_mean=events_registration_purchase_medium['time_dif'].mean() 
time_diff_medium_median=events_registration_purchase_medium['time_dif'].median() 
#print(events_registration_purchase_medium['time_dif'].describe())
time_diff_hard_mean=events_registration_purchase_hard['time_dif'].mean() 
time_diff_hard_median=events_registration_purchase_hard['time_dif'].median()
#print(events_registration_purchase_hard['time_dif'].describe())
print("Время между регистрацией и оплатой:       среднее                           медианное")
print('   для выбравших уровень easy:           ', time_diff_easy_mean, '       ', time_diff_easy_median)
print('   для выбравших уровень medium:         ', time_diff_medium_mean, '       ', time_diff_medium_median)
print('   для выбравших уровень hard:           ', time_diff_hard_mean, '       ', time_diff_hard_median)

Время между регистрацией и оплатой:       среднее                           медианное
   для выбравших уровень easy:            3 days 22:10:23.211640211         3 days 11:00:23
   для выбравших уровень medium:          4 days 06:12:06.576883384         4 days 03:35:26
   для выбравших уровень hard:            3 days 14:55:19.257918552         3 days 10:10:04.500000


**ВЫВОД 2:** Чем сложнее выбранный уровень (selected_level), тем более вероятен факт оплаты.
Временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности различается. При этом зависимость нелинейная. Наибольшее время между регистрацией и полатой проходит у групп с уровнем medium. Следует отметить что:
- медианное время меньше среднего во всех группах;
- у групп easy и hard медианное время практически совпадает.

### ПОДЫТОЖИМ ВЫВОДЫ.

**Вывод 1:** Чем сложнее выбранный уровень (selected_level), тем более вероятен факт оплаты.

**Вывод 2:** Чем сложнее выбранный уровень (selected_level), тем более вероятен факт оплаты.
Временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности различается. При этом зависимость нелинейная. Наибольшее время между регистрацией и полатой проходит у групп с уровнем medium. Следует отметить что:
- медианное время меньше среднего во всех группах;
- у групп easy и hard медианное время практически совпадает.